In [2]:
# !pip install -q transformers
# !pip install -q peft
# !pip install -q bitsandbytes
# !pip install -q torch

In [3]:
import torch
import os

In [4]:
# from google.colab import drive
# drive.mount('/content/drive')

# cache_dir = "/content/drive/MyDrive/Colab Notebooks/hf_models" # adding a cache dir is optional

In [5]:
cache_dir = "/home/hillary_kipkemoi/cache_dir" # adding a cache dir is optional

In [20]:
from transformers import LlamaTokenizer

checkpoint = "iocuydi/llama-2-amharic-3784m"
commit_hash = "04fcac974701f1dab0b8e39af9d3ecfce07b3773"
# The commit hash is needed, because the model repo was rearranged after this commit (files -> finetuned/files),
# and I couldn't load the model from the new structure

tokenizer = LlamaTokenizer.from_pretrained(checkpoint, revision=commit_hash, cache_dir= cache_dir)

In [19]:
embedding_size = tokenizer.get_input_embeddings().weight.shape[0]


AttributeError: 'LlamaTokenizer' object has no attribute 'get_input_embeddings'

In [7]:
print(tokenizer.encode("በ በአለማቀፍ ደረጃ የተፈፀመው የሞት ቅጣት"))

print(tokenizer.tokenize("በ በአለማቀፍ ደረጃ የተፈፀመው የሞት ቅጣት"))

[1, 32002, 32067, 32886, 34373, 32442, 43968, 50740, 38710, 35891]
['▁በ', '▁በአ', 'ለማ', 'ቀፍ', '▁ደረጃ', '▁የተፈፀመ', 'ው', '▁የሞት', '▁ቅጣት']


In [8]:
tokenizer.vocab_size

51008

In [9]:
import torch

torch.cuda.is_available() # Need GPU to use load_in_8bit

True

[source llama hf docs](https://huggingface.co/docs/transformers/main/en/model_doc/llama#transformers.LlamaForCausalLM)

In [21]:
from peft import PeftModel
from transformers import LlamaForCausalLM, GenerationConfig, AutoModelForSequenceClassification

llama_model = LlamaForCausalLM.from_pretrained(
    "meta-llama/Llama-2-7b-hf",
    load_in_8bit=True,
    device_map="auto",
    cache_dir= cache_dir, # optional
)

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


ValueError: Some modules are dispatched on the CPU or the disk. Make sure you have enough GPU RAM to fit the quantized model. If you want to dispatch the model on the CPU or the disk while keeping these modules in 32-bit, you need to set `load_in_8bit_fp32_cpu_offload=True` and pass a custom `device_map` to `from_pretrained`. Check https://huggingface.co/docs/transformers/main/en/main_classes/quantization#offload-between-cpu-and-gpu for more details. 

In [11]:
llama_model.resize_token_embeddings(len(tokenizer)) # needed because the fine-tuned model extended the tokenizer

Embedding(51008, 4096)

In [12]:
# this is the model we want:
model = PeftModel.from_pretrained(llama_model, "iocuydi/llama-2-amharic-3784m",revision =commit_hash, cache_dir= cache_dir)

In [15]:
model.is_quantized

True

AttributeError: 'LlamaForCausalLM' object has no attribute 'layers'

In [ ]:
# Clear GPU memory
torch.cuda.empty_cache()

# Shut down the notebook
os._exit(00)

NameError: name 'torch' is not defined